In [1]:
import os, json, tqdm
import numpy as np
import shutil
import cv2
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt

In [2]:
json_path = '/data2/chence/PanoHeadData/single_view_hq/dataset_v2_balanced_nohat.json'
src_parsing_dir = '/data2/chence/PanoHeadData/single_view_hq/align_parsing'
src_image_dir = '/data2/chence/PanoHeadData/single_view_hq/align_images'
with open(json_path, 'r') as f:
    dataset_json = json.load(f)

In [3]:
from multiprocessing import Pool
from tqdm import tqdm

def process_image(image_name):
    parsing_path_abs = os.path.join(src_parsing_dir, image_name.replace('jpg','png'))
    parsing_data = cv2.imread(parsing_path_abs, cv2.IMREAD_GRAYSCALE)
    image_size = parsing_data.shape[0] * parsing_data.shape[1]
    labels, counts = np.unique(parsing_data, return_counts=True)
    if 18 in labels: 
        ratio = counts[labels == 18] / image_size
        if ratio > 0.01: 
            # image_path_abs = os.path.join(src_image_dir, image_name)
            # image_data = cv2.imread(image_path_abs, cv2.IMREAD_COLOR)
            # fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            # ax[0].imshow(cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB))
            # ax[1].imshow(parsing_data)
            # plt.show()
            return image_name
    return None


all_keys = list(dataset_json.keys())
remove_names = []
with Pool(128) as p:
    result = list(tqdm(p.imap(process_image, all_keys), total=len(all_keys)))
result = np.array(result)



 37%|███▋      | 79929/214672 [00:24<00:40, 3304.87it/s]


KeyboardInterrupt: 

In [7]:
count = 0
dataset_json_copy = dataset_json.copy()
for r in result:
    if r is not None: 
        del dataset_json_copy[r]
        count += 1
print(count)
print(len(dataset_json_copy))
with open(json_path.replace('.json', '_nohat.json'), 'w') as f:
    json.dump(dataset_json_copy, f)


49617
214672
